In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from modules.pyspark import CPySpark

In [3]:
spark = CPySpark()

In [4]:
spark.sc

<SparkContext master=local appName=pyspark-shell>

Tạo ra một RDD từ list of words

In [5]:
lst_words = ["Spark", "is", "a", "framework", "for", "Big Data processing"]

In [6]:
rdd = spark.rdd(lst_words)

In [7]:
type(rdd)

pyspark.rdd.RDD

Tạo ra một RDD từ file dữ liệu

In [8]:
rdd2 = spark.rdd(file='./data/5000_points.txt', min_partitions=3)

In [9]:
type(rdd2)

pyspark.rdd.RDD

Lấy dòng đầu tiên trong data _(giống `.head()` của pandas.DataFrame)_

In [10]:
rdd2.first()

'664159\t550946'

Số vách ngăn tạo ra trong `rdd2`

In [11]:
rdd2.getNumPartitions()

3

Lấy từ URL

In [12]:
url = r'hdfs://bigdata.laptrinhpython.net:19000/t8.shakespeare.txt'

In [13]:
rdd3 = spark.rdd(file=url)

In [14]:
rdd3.take(10)

['This is the 100th Etext file presented by Project Gutenberg, and',
 'is presented in cooperation with World Library, Inc., from their',
 'Library of the Future and Shakespeare CDROMS.  Project Gutenberg',
 'often releases Etexts that are NOT placed in the Public Domain!!',
 '',
 'Shakespeare',
 '',
 '*This Etext has certain copyright implications you should read!*',
 '',
 '<<THIS ELECTRONIC VERSION OF THE COMPLETE WORKS OF WILLIAM']

<hr>

In [15]:
rdd = spark.rdd([1, 4, 3, 5, 6, 7])
rdd_map = rdd.map(lambda x: x**2)

rdd_map.collect()

[1, 16, 9, 25, 36, 49]

<hr>

In [16]:
rdd_filter = rdd.filter(lambda x: x > 3)
numbers_all = rdd_filter.collect()

numbers_all

[4, 5, 6, 7]

<hr>

In [17]:
rdd_string = spark.rdd(["Data Science", "Machine Learning", "Big Data"])
rdd_flatmap = rdd_string.flatMap(lambda x: x.split(' '))

rdd_flatmap.collect()

['Data', 'Science', 'Machine', 'Learning', 'Big', 'Data']

<hr>

In [18]:
rdd1 = spark.rdd([1, 2, 4, 4, 5])
rdd2 = spark.rdd([1, 3, 2, 1, 2])
rdd_union = rdd1.union(rdd2)

rdd_union.collect()

[1, 2, 4, 4, 5, 1, 3, 2, 1, 2]

In [19]:
rdd_union.count()

10

In [20]:
rdd_reduce = rdd_union.reduce(lambda x, y: x + y)

rdd_reduce

25

<hr>
Lưu RDD thành file

In [21]:
num_rdd = spark.rdd(range(100))

In [22]:
num_rdd.getNumPartitions()

1

Một folder mới dc tạo ra có tên là `number` trong folder `data`

In [23]:
num_rdd.saveAsTextFile('./data/number')

<hr>
Gôm tất cả partitions thành một rdd duy nhất, tức ko có vách ngăn

In [24]:
num_rdd.coalesce(1).saveAsTextFile('./data/number_all')

<hr>

In [26]:
rdd = spark.rdd([
    ('SV001', 8),
    ('SV002', 9),
    ('SV003', 10),
    ('SV002', 9),
    ('SV003', 9)
])

rdd_reduce = rdd.reduceByKey(lambda x, y: x + y)

for num in rdd_reduce.collect():
    print('{} có tổng điểm là {}'.format(num[0], num[1]))

SV001 có tổng điểm là 8
SV002 có tổng điểm là 18
SV003 có tổng điểm là 19


<hr>
sort by key

In [27]:
rdd_reduce_sort = rdd_reduce.sortByKey(ascending=False)

for num in rdd_reduce_sort.collect():
    print('{} có tổng điểm là {}'.format(num[0], num[1]))

SV003 có tổng điểm là 19
SV002 có tổng điểm là 18
SV001 có tổng điểm là 8


<hr>
groupByKey

In [31]:
rdd_group = rdd.groupByKey().collect()

for mssv, scores in rdd_group:
    print(mssv, list(scores))

SV001 [8]
SV002 [9, 9]
SV003 [10, 9]


<hr>
join

In [32]:
rdd_math = spark.rdd([('SV001', 8), ('SV002', 9), ('SV003', 9)])
rdd_english = spark.rdd([('SV001', 9), ('SV002', 8), ('SV003', 8)])

rdd_join = rdd_math.join(rdd_english)

In [33]:
rdd_join.collect()

[('SV002', (9, 8)), ('SV001', (8, 9)), ('SV003', (9, 8))]

In [34]:
rdd_math = spark.rdd([('SV001', 8), ('SV002', 9), ('SV003', 9)])
rdd_english = spark.rdd([('SV001', 9), ('SV002', 8)])

rdd_join = rdd_math.join(rdd_english)

In [35]:
rdd_join.collect()

[('SV002', (9, 8)), ('SV001', (8, 9))]

<hr>

countByKey()

In [37]:
rdd = spark.rdd([
    ('SV001', 8),
    ('SV002', 9),
    ('SV003', 10),
    ('SV002', 9),
    ('SV003', 9)
])

for key, val in rdd.countByKey().items():
    print(key, val)

SV001 1
SV002 2
SV003 2


<hr>
collectAsMap()

In [38]:
rdd = spark.rdd([
    ('SV001', 8),
    ('SV002', 9),
    ('SV001', 10),
    ('SV002', 6),
    ('SV003', 9),
    ('SV003', 8)
])